In [ ]:
# %load faqbot.py
#!/usr/bin/env python

# In[2]:


#!/usr/bin/python
# -*- coding: utf-8 -*-
from tkinter import *
from tkinter import messagebox
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from difflib import SequenceMatcher
from nltk.tokenize import RegexpTokenizer
import copy
import random
import webbrowser
import nltk
import numpy as np
import random
import string # to process standard python strings
import subprocess
import sys
import psutil
import os
import psutil
#Madhu modifications start
from scipy import spatial
#Madhu modifications start
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# In[3]:


#One Time download
#nltk.download('punkt')
#nltk.download('wordnet')


# In[4]:


f=open('output_desc.csv','r')
raw=f.read()
raw=raw.lower()# converts to lowercase

sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words

lemmer = nltk.stem.WordNetLemmatizer()

#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):

    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

def get_lable(line_num):
    with open('output_lable.csv', 'r') as content_file:
        content = content_file.read()
    content = content.split('\n')
    for line in content:
        if str(line_num) == (line.split(',')[0]):
            return line.split(',')[1]

def get_link(line_num):
    with open('output_link.csv', 'r') as content_file:
        content = content_file.read()
    content = content.split('\n')
    for line in content:
        if line_num+"," in line:
            #print (line.split(',')[0])
            return get_lable(line.split(',')[0]),line.split(',')[1]

def show_link(label,link):
    values_arr=[]
    def click1():
        webbrowser.open(link)
    message.see("end")
    #message.insert("end", label+'\n', hyperlink.add(click1))
    message.insert("end", label+'\n', hyperlink.add(click1))
    message.see("end")



# In[ ]:


#Finding documents only for the requested  file formats
#.docx, .xlsx, doc, .zip, .ppt, .pptx, .pdf, .xlsm
#remove these from output
#.db, .tmp,
#.wmv, .mp4


# In[52]:


def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    #vals = 1 - spatial.distance.cosine(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    sent_tokens.remove(user_response)
    #print(flat)
    req_tfidf = flat[-2]
    if(req_tfidf>=0.75 or sent_tokens==None):
        robo_response=robo_response+"Bot: I am sorry! I don't understand you"
        return robo_response
    else:
        resp = ""
        #dict1 = {"A":get_link(sent_tokens[idx].split(",")[0]),"B":get_link(sent_tokens[idx-1].split(",")[0]),"C":get_link(sent_tokens[idx-2].split(",")[0])}
        dict1 = {}
        try:
            #print(sent_tokens[idx-i])
            message.insert("end", "Bot : Here's the list of relevant link(s) that could be found\n","bot")
            for i in range(1,6):
                label,link = get_link(sent_tokens[idx-i].split(",")[0])
                #print(label)
                #if re.search(pattern, string, flags=0)

                message.insert("end",i,"bot")
                show_link(label,link)
        except:
            robo_response=robo_response+"Bot: I am sorry! I don't understand you"
            return robo_response
        #print(dict1)
        return (resp)
        #return robo_response


# In[6]:


#FIX-ME update vocabulary with the word
vocabulary = [    'pertain',    'more',    'fetch',    'be',    'there',    'do',    'you',    'have',    'any',    'is',
              'my',    'on',    'can',    'i',    'get',    'some',    'am',    'look',    'for',    'the',    'to',
              'share',    'me',    'of',    'please',    'a',    'very',    'at',    'with',    'relate',    'sorry',
              'informatica',    'ab initio',    'talend',    'etl',    'dwt',    'dwh',    'rfp',    'tableu',    'case',
              'study',    'video',    'doccument',    'link',    'artefact',    'hello',    'hi',    'train'    ]


def lemmatize(text):
    """Function to lemmatize a text string.
    Lemmatizing is the process of removing 'ing', 'ly', 'es' etc. suffixes from the test string """

    lem = WordNetLemmatizer()
    return ' '.join(list(map(lambda x: lem.lemmatize(x, 'v'),
                    text.split())))


def check_spellings(text):
    """Function to check for typos in the text string.
    It corrects the spelling if it finds some spelling matching to 85% of the lexicon used in the vocabulary"""

    for word in vocabulary:
        text = correct(word, text, 0.90)
    return text


def correct(search_key, text, strictness):
    """This function corrects the spelling in case there is a typo resembling to 85% of the spelling of the matching word"""

    text_copy = copy.deepcopy(text)
    words = text.split()
    for word in words:
        similarity = SequenceMatcher(None, word, search_key)
        if similarity.ratio() > strictness:
            text_copy = text_copy.replace(word, search_key)
    return text_copy


def remove_noise(text):
    """This function is used to filter out the words that are noises and do not convey any important message"""

    text = text.split()
    word = [word for word in text if word not in [
        'pertain',
        'estimate',
        'link',
        'and',
        'more',
        'fetch',
        'be',
        'there',
        'do',
        'you',
        'have',
        'any',
        'is',
        'my',
        'on',
        'can',
        'i',
        'get',
        'some',
        'am',
        'look',
        'for',
        'the',
        'to',
        'share',
        'me',
        'of',
        'please',
        'a',
        'very',
        'at',
        'with',
        'relate',
        'sorry',
        'train',
        'training'
        ]]
    return ' '.join(word)

def standardize(text):
    lookup_dict = {
        'rt': 'Retweet',
        'dm': 'direct message',
        'awsm': 'awesome',
        'luv': 'love',
        'doc': 'document',
        'lvl': 'level',
        'plz': 'please',
        'pls': 'please',
        'info': 'information',
        'u': 'you',
        'sry': 'sorry',
        'gr8': 'great',
        'ppt': 'presentation',
        'vid': 'video',
        'use': 'study',
        'material': 'document',
        'train': 'document',
        'training': 'document'
        }
    new_words = []
    for word in text.split():
        if word in lookup_dict:
            new_words.append(lookup_dict[word])
            continue
        new_words.append(word)
    return ' '.join(new_words)

def prepare_utterance(line):
    """Function to prepare the utterance to a standard text string
    1. Creates deep copy of utterance
    2. Trims the string
    2. Converts text to lower string
    3. Removes punctuations for the text"""

    text = copy.deepcopy(line).strip().lower()
    tokenizer = RegexpTokenizer(r'\w+')
    text = ' '.join(tokenizer.tokenize(text))
    text = standardize(text)
    text = check_spellings(text)
    text = lemmatize(text)
    return text

def listen():
    utterance = input()
    return (utterance)

def prepare_reply():
    pass

def entities_found():
    return True

def confirmed(utterance):
    return True

def fetch_links(entities):
    return ["www.google.com"]

def show_what_found(links):
    print(links)

def clear_entity():
    pass

def discover_entities():
    pass

def apologize():
    print("Sorry I do not understand this. Please elaborate")

def get_bussiness_utterance():
    #check if the utterance is not casual

    utterance = listen()
    while casual_utterance(utterance):
        utterance = listen()
    return utterance


# In[7]:


def casual_utterance(sentence,message,input_value):
    '''If utterance is casual reply back accordingly'''

    GREETING_WC_INPUTS = ("hello", "hi", "greetings","hey")
    GREETING_WC_RESPONSES = ["Hi", "Hey", "*nods*", "Hi there", "Hello", "I am glad! You are talking to me"]
    GREETING_INPUTS = ("bye", "see you", "byebye","bye bye")
    GREETING_RESPONSES = ["Hi", "Hey", "*nods*", "Hi there", "Hello", "I am glad! You talked to me"]
    for word in sentence.split():
        if word.lower() in GREETING_WC_INPUTS:
            #print(random.choice(GREETING_RESPONSES))
            message.insert("end","Bot: "+random.choice(GREETING_WC_RESPONSES)+"\n","bot")
            message.see("end")
            #print("Hi user, What can I find for you?")
            message.insert("end","Bot: "+"Hi user, What can I find for you?"+"\n","bot")
            message.see("end")
            return True
        if word.lower() in GREETING_INPUTS:
            #print(random.choice(GREETING_RESPONSES))
            message.insert("end","Bot: "+random.choice(GREETING_RESPONSES)+"\n","bot")
            message.see("end")
            #print("Hi user, What can I find for you?")
            message.insert("end","Bot: "+"Hi user, Thank you for using my service. Hope you found the releavant information."+"\n","bot")
            message.see("end")
            return True

    WORKQUES_INPUTS = ("sup","sup?", "what's up?")
    WORKQUES_RESPONSES = ["Not much"]
    for word in sentence.split():
        if word.lower() in WORKQUES_INPUTS:
            #print(random.choice(WORKQUES_RESPONSES))
            message.insert("end","Bot: "+random.choice(WORKQUES_RESPONSES)+"\n","bot")
            message.see("end")
            #print("Just looking up some documents for curious souls :)")
            message.insert("end","Bot: "+"Just looking up some documents for curious souls :)"+"\n","bot")
            message.see("end")
            return True

    THANK_INPUTS = ("thanks", "that's great", "that was helpful")
    THANK_RESPONSES = ["you are welcome", "happy to help :D"]
    for word in sentence.split():
        if word.lower() in THANK_INPUTS:
            #print (random.choice(THANK_RESPONSES))
            message.insert("end","Bot: "+random.choice(THANK_RESPONSES)+"\n","bot")
            message.see("end")
            return True
    return False


# In[8]:


def on_closing():

    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        GUI.destroy()


# In[9]:


def Enter_Hit(event):

    utterance=input_user.get()
    message.tag_config("bot",foreground="black")
    message.tag_config("user", foreground="blue")
    utter=copy.deepcopy(utterance)

    utter = prepare_utterance(utter)
    #print(utter)
    message.config(state=NORMAL,fg="Blue")
    message.insert("end","User: "+utterance+"\n","user")
    message.see("end")
    if casual_utterance(utterance,message,input_value):
        input_value.set("")
        message.config(state=DISABLED)
    else:
        resp = response(utter)
        message.insert("end",resp+"\n","bot")
        message.see("end")
        input_value.set("")
        message.config(state=DISABLED)


# In[10]:


class HyperlinkManager:

    def __init__(self, text):

        self.text = text
        self.text.tag_config("hyper", foreground="blue", underline=1)
        self.text.tag_bind("hyper", "<Enter>", self._enter)
        self.text.tag_bind("hyper", "<Leave>", self._leave)
        self.text.tag_bind("hyper", "<Button-1>", self._click)
        self.reset()

    def reset(self):
        self.links = {}

    def add(self, action):
        # add an action to the manager.  returns tags to use in
        # associated text widget
        tag = "hyper-%d" % len(self.links)
        self.links[tag] = action
        return "hyper", tag

    def _enter(self, event):
        self.text.config(cursor="hand2")

    def _leave(self, event):
        self.text.config(cursor="")

    def _click(self, event):
        for tag in self.text.tag_names(CURRENT):
            if tag[:6] == "hyper-":
                self.links[tag]()
                return


# In[12]:


def is_running(script):
    for q in psutil.process_iter():
        #if q.name().startswith('myapp'):
            #if len(q.cmdline())>1 and script in q.cmdline()[1] and q.pid !=os.getpid():
            if script in q.name() and q.pid !=os.getpid():
                #print("'{}' Chatbot window is already opened".format(script))
                return True

    return False


# In[13]:


#import mtTinkter


# In[14]:


###-Nav-Edit
if __name__ == '__main__':

    if not is_running("Chatbot.exe"):
        patience = 4
        entities = []

        #print("Application starts...")
        '''
        while True:
            if patience > 0:
                #if utterance is query?
                utterance =  get_bussiness_utterance()
                if entities_found():
                    reply = prepare_reply()
                    print(reply)
                    utterance = listen()
                    if confirmed(utterance):
                        links = fetch_links(entities)
                        show_what_found(links)
                    else:
                        clear_entity()
                        patience = patience - 1

            else:
                apologize()
                links = fetch_links(entities)
                show_what_found(links)            
                clear_entity()
                patience = 4
        '''
        #checkrunning()
        GUI=Tk()
        GUI.title("Infosys CoE portal Searchbot")
        input_value=StringVar()
        scrollbar = Scrollbar(GUI)
        scrollbar.pack(side=RIGHT, fill=Y)
        label1=Label(GUI,text="Welcome to CoE Portal!",fg="Red")
        label1.pack(side=TOP,fill=X)
        message=Text(GUI)
        message.pack()
        hyperlink = HyperlinkManager(message)
        message.config(state=DISABLED,yscrollcommand=scrollbar.set)
        scrollbar.config(command=message.yview)

        input_user=Entry(GUI,text=input_value)
        user_label=Label(GUI,text="Enter message here:",fg="Red")
        user_label.pack(fill=X)
        input_user.pack(fill=X)
        input_user.bind("<Return>", Enter_Hit)
        GUI.protocol("WM_DELETE_WINDOW", on_closing)
        #GUI.protocol("WM_DELETE_WINDOW", quit)
        GUI.resizable(False, False)

        # Gets the requested values of the height and widht.
        #windowWidth = 600 #GUI.winfo_screenwidth()
        #windowHeight = 450 #GUI.winfo_screenheight()

        # Gets both half the screen width/height and window width/height
        positionRight = int(GUI.winfo_screenwidth()/2 - GUI.winfo_reqwidth()/2)
        positionDown = int(GUI.winfo_screenheight()/2 - GUI.winfo_reqheight()/2)
        #print(GUI.winfo_screenwidth())
        #print(GUI.winfo_screenheight())
        #print(GUI.winfo_reqwidth())
        #print(GUI.winfo_reqheight())
        #print(positionRight)
        #print(positionDown)

        # Positions the window in the center of the page.
        #GUI.geometry("+{}+{}".format(positionRight, positionDown))
        w = int(positionRight - GUI.winfo_reqwidth())
        h = int(positionDown - GUI.winfo_reqheight())
        #print(h)
        #print(w)
        GUI.geometry("+{}+{}".format(w, h))
        #GUI.geometry('%dx%d+%d+%d' % (600, 450, positionDown, positionRight))
        GUI.mainloop()

    else:
        messagebox.showerror("Chatbot Error", "Chatbot instance is already running")
        sys.exit()


# In[ ]:


# if is_running("Chatbot.exe"):
    # messagebox.showerror("Chatbot Error", "Chatbot instance is already running")
    # sys.exit()


# def running():
    # n=0# number of instances of the program running
    # prog=[line.split() for line in subprocess.check_output("tasklist").splitlines()]
    # [prog.pop(e) for e in [0,1,2]] #useless
    # for task in prog:
        # if task[0]=="Chatbot.exe":
            # n=n+1
    # if n>1:
        # return True
    # else:
        # return False

# if running():
    # messagebox.showerror("Chatbot Error", "Chatbot instance is already running")
    # sys.exit()


if __name__ == '__main__':

    if not is_running("Chatbot.exe"):
        patience = 4
        entities = []

        #print("Application starts...")
        '''
        while True:
            if patience > 0:
                #if utterance is query?
                utterance =  get_bussiness_utterance()
                if entities_found():
                    reply = prepare_reply()
                    print(reply)
                    utterance = listen()
                    if confirmed(utterance):
                        links = fetch_links(entities)
                        show_what_found(links)
                    else:
                        clear_entity()
                        patience = patience - 1

            else:
                apologize()
                links = fetch_links(entities)
                show_what_found(links)            
                clear_entity()
                patience = 4
        '''
        #checkrunning()
        GUI=Tk()
        GUI.title("Infosys CoE portal Searchbot")
        input_value=StringVar()
        scrollbar = Scrollbar(GUI)
        scrollbar.pack(side=RIGHT, fill=Y)
        label1=Label(GUI,text="Welcome to CoE Portal!",fg="Red")
        label1.pack(side=TOP,fill=X)
        message=Text(GUI)
        message.pack()
        hyperlink = HyperlinkManager(message)
        message.config(state=DISABLED,yscrollcommand=scrollbar.set)
        scrollbar.config(command=message.yview)
        input_user=Entry(GUI,text=input_value)
        user_label=Label(GUI,text="Enter message here:",fg="Red")
        user_label.pack(fill=X)
        input_user.pack(fill=X)
        input_user.bind("<Return>", Enter_Hit)
        GUI.protocol("WM_DELETE_WINDOW", on_closing)
        #GUI.protocol("WM_DELETE_WINDOW", quit)
        GUI.resizable(False, False)

        # Gets the requested values of the height and widht.
        #windowWidth = 600 #GUI.winfo_screenwidth()
        #windowHeight = 450 #GUI.winfo_screenheight()

        # Gets both half the screen width/height and window width/height
        positionRight = int(GUI.winfo_screenwidth()/2 - GUI.winfo_reqwidth()/2)
        positionDown = int(GUI.winfo_screenheight()/2 - GUI.winfo_reqheight()/2)
        #print(GUI.winfo_screenwidth())
        #print(GUI.winfo_screenheight())
        #print(GUI.winfo_reqwidth())
        #print(GUI.winfo_reqheight())
        #print(positionRight)
        #print(positionDown)

        # Positions the window in the center of the page.
        #GUI.geometry("+{}+{}".format(positionRight, positionDown))
        w = int(positionRight - GUI.winfo_reqwidth())
        h = int(positionDown - GUI.winfo_reqheight())
        #print(h)
        #print(w)
        GUI.geometry("+{}+{}".format(w, h))
        #GUI.geometry('%dx%d+%d+%d' % (600, 450, positionDown, positionRight))
        GUI.mainloop()

    else:
        messagebox.showerror("Chatbot Error", "Chatbot instance is already running")
        sys.exit()


# In[ ]: